In [1]:
# import libereries
import os 
import json,pickle
import numpy as np 
from sklearn.pipeline import Pipeline

# import  libereries for braries for data structuring and analysis(data visualization)
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.stats import chi2_contingency,chi2
#import warnings to avoid warnings
import warnings
warnings.filterwarnings('ignore')

# importing encoders
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder,LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
# importing randomised cross validation and train test split
from sklearn.model_selection import train_test_split,KFold,cross_val_score,RandomizedSearchCV
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import KBinsDiscretizer 
# importing Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
#from xgboost import XGBClassifier as xgb
from sklearn.feature_selection import SelectKBest, chi2
# importing performance matrixes
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,roc_curve
from sklearn.metrics import classification_report

from sklearn.metrics import confusion_matrix

# importing SMOTENN to resample data(balancing the data)
#from imblearn.combine import SMOTEENN

# setting to display max columns 
pd.set_option("display.max_columns",None)


Matplotlib is building the font cache; this may take a moment.


ModuleNotFoundError: No module named 'seaborn'

In [2]:
pip install seaborn 

ERROR: Could not find a version that satisfies the requirement seaborn (from versions: none)
ERROR: No matching distribution found for seaborn
Note: you may need to restart the kernel to use updated packages.


In [4]:
data=pd.read_csv('bank.csv',delimiter =';')
data.head()

In [ ]:
data=data.drop(['contact', 'day', 'poutcome'], axis=1)
data.head()

In [ ]:
for i in data.columns:
    data[i]=np.where(data[i]=="unknown",np.nan,data[i])
    
data.isnull().sum()

In [ ]:
data.head()

In [ ]:
# numerical data
num_data=data.select_dtypes(exclude='object')
num_data.head()

In [ ]:
X_train, X_test, y_train, y_test =train_test_split (data.drop('y',axis=1),data[['y']], test_size=0.2,random_state=42)
X_train.shape, X_test.shape 

In [ ]:
X_train.head() 

In [ ]:
(X_train.age).plot()

In [ ]:
X_train

In [ ]:
y_train_labeled=LabelEncoder().fit_transform(y_train)
y_train_labeled

In [ ]:
y_test_labeled=LabelEncoder().fit_transform(y_test) 
y_test_labeled

In [ ]:
len(data.job.unique()) 

In [ ]:
#imputation
# Imputation
trf1=ColumnTransformer([('impute_job',SimpleImputer(strategy='most_frequent' ), [1]) ,('impute_edu',SimpleImputer(strategy='most_frequent'), [3])],remainder='passthrough')


In [ ]:
# Oridinal Encoding 
trf2=ColumnTransformer([('ord_martial_education_default_housing_loan',OrdinalEncoder(categories=[['single','married', 'divorced'], ['primary','secondary', 'tertiary'], ['no', 'yes'], ['no', 'yes'], ['no', 'yes']],dtype=int),[3,1,4,6,7])],remainder ='passthrough') 

In [ ]:
#OneHotEncoding
trf3=ColumnTransformer([('ohe_job_month',OneHotEncoder(sparse=False,handle_unknown='ignore', dtype=int,drop='first'), [5, 8])],remainder='passthrough')

In [ ]:
#Scaling
trf4=ColumnTransformer([('Power_transform', PowerTransformer(),[26,27,28,29,30,31])], remainder='passthrough' )

In [ ]:
#Feature Selection
trf5=SelectKBest(score_func=chi2,k=30)

In [ ]:
# train model
trf6=RandomForestClassifier() 

In [ ]:
pipe=Pipeline([('trf1', trf1),('trf2', trf2), ('trf3', trf3), ('trf4', trf4),
               ('trf6', trf6) ])


In [ ]:
trf7=LogisticRegression() 

In [ ]:
pipe1=Pipeline([('trf1', trf1),('trf2', trf2), ('trf3', trf3), ('trf4', trf4),
               ('trf7', trf7) ])


In [ ]:
# Display Pipeline
from sklearn import set_config
set_config(display='diagram') 

In [ ]:
pipe.fit(X_train,y_train_labeled)

In [ ]:
pipe1.fit(X_train,y_train_labeled) 

In [ ]:
pred_lr=pipe1.predict(X_test) 
pred_lr

In [ ]:
print(classification_report(y_test_labeled,pred_lr,target_names=['no', 'yes'])) 

In [ ]:
confusion_matrix(y_test_labeled, pred_lr, labels=[0, 1]) 

In [ ]:
pred=pipe.predict(X_test)
pred

In [ ]:
print(classification_report(y_test_labeled, pred, target_names=['no', 'yes'] ) ) 

In [ ]:
confusion_matrix(y_test_labeled, pred, labels=[0, 1]) 

In [ ]:
print('mean score of logistics  model  ={}'.format(np.mean(cross_val_score(pipe1,X_train,y_train_labeled, cv=10,scoring='accuracy')))) 

In [ ]:
print('mean score of random forest model  ={}'.format(np.mean(cross_val_score(pipe,X_train,y_train_labeled, cv=10,scoring='accuracy')))) 

In [ ]:
X_train_impute=pd.DataFrame(trf1.fit_transform(X_train))

In [ ]:
X_train_ord=pd.DataFrame(trf2.fit_transform(X_train_impute))

In [ ]:
X_train_impute

In [ ]:
X_train_ord

In [ ]:
X_train_ohe=pd.DataFrame(trf3.fit_transform(X_train_ord))
X_train_ohe

In [ ]:
ohe=OneHotEncoder(sparse=False, handle_unknown='ignore', dtype=int,drop='first' )
ohe.fit_transform(X_train_ord[[5]]).shape

In [ ]:
X_train_ohe[[26,27,28,29,30,31]]=X_train_ohe[[26, 27,28,29,30,31]].astype(int) 
X_train_ohe.head()

In [ ]:
X_train_func_tran=pd.DataFrame(trf4.fit_transform(X_train_ohe))
#X_train_func_tran=X_train_func_tran.drop(4,axis=1)
X_train_func_tran 

In [ ]:
trf6.fit(X_train_func_tran,y_train_labeled) 

In [ ]:
for i in X_train_func_tran.columns:
    print('{}'.format(i)+'=',X_train_func_tran[i].skew())

In [ ]:
# defining a function to get distribution and outliers
def get_distribution(data):
    fig, axes = plt.subplots(1, 2, figsize=(16,4))
    sns.distplot(data, hist = False,ax = axes[0],color="r")
    sns.boxplot(data,ax = axes[1],color="g")
    return plt.show()

In [ ]:

for i in X_train_func_tran.columns:
    get_distribution(X_train_func_tran[i])
    

In [ ]:
kbins=KBinsDiscretizer(n_bins=7, encode='ordinal',strategy='quantile')
trf_Kbins=ColumnTransformer([('binning', kbins,[0])],remainder ='passthrough')


In [ ]:
X_train_kbins=pd.DataFrame(trf_Kbins.fit_transform(X_train))

X_train_kbins.rename(columns={0:'age_trf'} ,inplace=True)
X_train_kbins.head()

In [ ]:
age=X_train_kbins['age_trf'] 
age

In [ ]:
X_train_kbins.drop(columns='age_trf', axis=1, inplace=True)
X_train_kbins.head() 

In [ ]:
print('mean score of logistics  model  ={}'.format(np.mean(cross_val_score(pipe1,X_train_kbins,y_train_labeled, cv=10,scoring='accuracy')))) 

In [ ]:
print('mean score of random forest model  ={}'.format(np.mean(cross_val_score(pipe,X_train_kbins,y_train_labeled, cv=10,scoring='accuracy')))) 

In [ ]:

X_train_age_agetrf=pd.concat([X_train.age,X_train_kbins.age_trf],axis=1)


X_train_age_agetrf['age_range']= pd.cut(x=X_train['age'],
                                    bins=trf_Kbins.named_transformers_['binning'].bin_edges_[0].tolist()) 
#pd.cut(x=X_train['age'], bins=trf_Kbins.named_transformers_('binning_age').bin_edges_[0].tolist()) 
X_train_age_agetrf.head()